In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import random
import collections
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split

from torch.autograd import Variable #自動微分用
import torch.nn as nn #ネットワーク構築用
import torch.optim as optim #最適化関数
import torch.nn.functional as F #ネットワーク用の様々な関数
import torch.utils.data #データセット読み込み関連
import torchvision #画像関連
import util

In [2]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
 
        self.hidden_size = hidden_size
 
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
 
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
 
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [3]:
dataloader = util.dataloader()
dataloader.normalize()
X_train, X_test, y_train, y_test = dataloader.dataset() 

sample_nums: [(8, 2206), (6, 1364), (18, 1022), (4, 894), (17, 807), (10, 649), (19, 512), (13, 489), (11, 476), (3, 444), (1, 397), (12, 372), (2, 305), (5, 228), (15, 203), (14, 186), (9, 172), (20, 167), (16, 162), (7, 128)]
min_num: 128
クラス8 削除サンプル数: 2078 (94.20％)
クラス6 削除サンプル数: 1236 (90.62％)
クラス18 削除サンプル数: 894 (87.48％)
クラス4 削除サンプル数: 766 (85.68％)
クラス17 削除サンプル数: 679 (84.14％)
クラス10 削除サンプル数: 521 (80.28％)
クラス19 削除サンプル数: 384 (75.00％)
クラス13 削除サンプル数: 361 (73.82％)
クラス11 削除サンプル数: 348 (73.11％)
クラス3 削除サンプル数: 316 (71.17％)
クラス1 削除サンプル数: 269 (67.76％)
クラス12 削除サンプル数: 244 (65.59％)
クラス2 削除サンプル数: 177 (58.03％)
クラス5 削除サンプル数: 100 (43.86％)
クラス15 削除サンプル数: 75 (36.95％)
クラス14 削除サンプル数: 58 (31.18％)
クラス9 削除サンプル数: 44 (25.58％)
クラス20 削除サンプル数: 39 (23.35％)
クラス16 削除サンプル数: 34 (20.99％)
クラス7 削除サンプル数: 0 (0.00％)


In [4]:
n_hidden = 200
model = RNN(30, n_hidden, dataloader.label_count())

In [5]:
train = torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
train_loader = torch.utils.data.DataLoader(train, batch_size=100, shuffle=True)
test = torch.utils.data.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
test_loader = torch.utils.data.DataLoader(test, batch_size=100, shuffle=True)

In [6]:
criterion = nn.NLLLoss()

learning_rate = 0.005
hidden = model.initHidden()

In [7]:
#トレーニング
#エポック数の指定
for epoch in range(2):  # loop over the dataset multiple times
    #データ全てのトータルロス
    running_loss = 0.0 

    for i, data in enumerate(train_loader):
        #入力データ・ラベルに分割
        # get the inputs
        inputs, labels = data

        # Variableに変形
        # wrap them in Variable
        inputs, labels = Variable(inputs.float()), Variable(labels)

        # optimizerの初期化
        # zero the parameter gradients
        model.zero_grad()

        output, hidden = model(inputs, hidden)

        loss = criterion(output, input)
        loss.backward()

        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: invalid argument 0: Tensors must have same number of dimensions: got 3 and 2 at /pytorch/aten/src/TH/generic/THTensorMath.c:3577